In [1]:
##### Star Importing / Initializing

In [2]:
import pyspark as s

sc = s.SparkContext.getOrCreate()

print(sc)
print(sc.version)


23/11/18 03:33:35 WARN Utils: Your hostname, Scotland-Yard resolves to a loopback address: 127.0.1.1; using 192.168.34.193 instead (on interface wlo1)
23/11/18 03:33:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/18 03:33:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=pyspark-shell>
3.5.0


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Code").getOrCreate()

print(spark)

In [4]:
import pandas as pd
import numpy as np

pd_temp = pd.DataFrame(np.random.random(10))

spark_temp = spark.createDataFrame(pd_temp)

print(spark.catalog.listTables())

spark_temp.createOrReplaceTempView('temp')
print(spark.catalog.listTables())

/home/metaphysicist/Coding/Projects/Data_Projects/Analyze_Flow/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


[]
[Table(name='temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]


##### Load


In [5]:
data_path = "Legally_Operating_Businesses.csv"

businesses = spark.read.csv(data_path,header=True)

businesses.show(10)


23/11/18 03:33:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+------------+-----------------------+--------------+---------------------+--------------------+-------------------+--------------------+----------------+-------------------+-----------------------------+-------------+----------------+-----------+--------------------+---------------+------------+---------------+----------------+----+----+----+------------+------+---------+--------+--------+
|DCA License Number|License Type|License Expiration Date|License Status|License Creation Date|            Industry|      Business Name|     Business Name 2|Address Building|Address Street Name|Secondary Address Street Name| Address City|   Address State|Address ZIP|Contact Phone Number|Address Borough|Borough Code|Community Board|Council District| BIN| BBL| NTA|Census Tract|Detail|Longitude|Latitude|Location|
+------------------+------------+-----------------------+--------------+---------------------+--------------------+-------------------+--------------------+----------------+-

In [6]:
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/home/metaphysicist/Coding/Projects/Data_Projects/Analyze_Flow/spark-warehouse')]

In [7]:
businesses.columns

['DCA License Number',
 'License Type',
 'License Expiration Date',
 'License Status',
 'License Creation Date',
 'Industry',
 'Business Name',
 'Business Name 2',
 'Address Building',
 'Address Street Name',
 'Secondary Address Street Name',
 'Address City',
 'Address State',
 'Address ZIP',
 'Contact Phone Number',
 'Address Borough',
 'Borough Code',
 'Community Board',
 'Council District',
 'BIN',
 'BBL',
 'NTA',
 'Census Tract',
 'Detail',
 'Longitude',
 'Latitude',
 'Location']

##### Transform


In [8]:
selected_cols = businesses.select('License Type',
 'License Expiration Date',
 'License Status',
 'License Creation Date',
 'Industry',
 'Business Name',
 'Address City'
 )


In [9]:
from pyspark.sql.functions import datediff, to_date,  months_between,col,round, cast

selected_cols = selected_cols.withColumn('License_In_Month',round(months_between(
        to_date('License Expiration Date',"MM/dd/yyyy"),
        to_date('License Creation Date',"MM/dd/yyyy")).cast("int"))
        )

selected_cols = selected_cols.withColumn('License_In_Years',round(col("License_In_Month")/12).cast("int"))

selected_cols = selected_cols.drop("License_Duration","License_In_Month")
# print(help(col))
# val = selected_cols.select('License Creation Date')
# print(type(val)) 
selected_cols.show(10)

+------------+-----------------------+--------------+---------------------+--------------------+-------------------+-------------+----------------+
|License Type|License Expiration Date|License Status|License Creation Date|            Industry|      Business Name| Address City|License_In_Years|
+------------+-----------------------+--------------+---------------------+--------------------+-------------------+-------------+----------------+
|    Business|             02/28/2017|      Inactive|           02/04/2010|Home Improvement ...|BARBARINO, JOHN JR.|    MANALAPAN|               7|
|  Individual|             03/31/2024|        Active|           01/11/2022|   Sightseeing Guide|    Nieciak, Joseph|     BROOKLYN|               2|
|  Individual|             03/31/2024|        Active|           10/31/2017|   Sightseeing Guide|      Arcaro, Tyler|     NEW YORK|               6|
|  Individual|             03/31/2024|        Active|           09/11/2018|   Sightseeing Guide|     HESSE, DAPH

In [11]:
# selected_cols.describe().show()
selected_cols = selected_cols.drop_duplicates()
selected_cols.dtypes
selected_cols.printSchema()
# write analytics of dataframe selected_cols to csv file


root
 |-- License Type: string (nullable = true)
 |-- License Expiration Date: string (nullable = true)
 |-- License Status: string (nullable = true)
 |-- License Creation Date: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Business Name: string (nullable = true)
 |-- Address City: string (nullable = true)
 |-- License_In_Years: integer (nullable = true)

